# hand_strength

> Fill in a module description here

In [ ]:
#|default_exp hand_strength

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|export
from fastcore.all import *
import polvo as pv
import truco as tr
import numpy as np
import random

/home/lgvaz/miniconda3/envs/truco/lib/python3.10/site-packages/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#|export
def compare(hand1, hand2, round_rank):
    ranks1 = tr.cards2ranks(hand1, round_rank)
    ranks2 = tr.cards2ranks(hand2, round_rank)
    ranks1 = np.sort(ranks1)[-1:-4:-1]
    ranks2 = np.sort(ranks2)[-1:-4:-1]
    return (ranks1 - ranks2)

In [ ]:
#|export
def simulate(hand, round_rank, card_pool):
    hand2 = random.sample(card_pool, 6)
    return compare(hand, hand2, round_rank)

In [ ]:
#|export
class TaskWrapper:
    def __init__(self, fn, *args, **kwargs):
        self.fn, self.args, self.kwargs = fn, args, kwargs
        
    def __call__(self, _):
        return self.fn(*self.args, **self.kwargs)

In [ ]:
#|export
def monte_carlo(hand, round_rank, card_pool, n_simulations):
    return np.array(parallel(TaskWrapper(simulate, hand, round_rank, card_pool), range(n_simulations)))

In [ ]:
#|export
def hand_stregth(hand, faceup, n_simulations=2000):
    card_pool = tr.round_card_pool(hand, faceup)
    round_rank = tr.round_rank(faceup)
    
    results = monte_carlo(hand, round_rank, card_pool, n_simulations)
    results_sign = np.sign(results)
    wins = np.sign(results_sign.sum(axis=1))
    score, counts = np.unique(wins, return_counts=True)
    return {s: c/len(results) for s, c in zip(score, counts)}

In [ ]:
hand = '5d 3h qs'.split()
faceup = '4d'

In [ ]:
hand_stregth(hand, faceup)

{-1: 0.402, 0: 0.063, 1: 0.535}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()